This program executes one simulation using initial parameters that were set in 'main_code.ipynb' and are saved and imported from 'initial_parameters.py'.

# Import libraries

In [1]:
import hoomd
import hoomd.md

import numpy as np
import math
import os

#import file with initial parameters
import initial_parameters

# Initialize the system

In [2]:
hoomd.context.initialize("")

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


# Define the parameters (that were set in 'main_code.ipynb')

In [3]:
#define number of polymers segments (monomers)
Monomers = initial_parameters.Monomers

#define number of bonds between segments
Bonds = initial_parameters.Bonds

#define spring constant (Rouse model) for bonds between segments in units energy/distance^2
k = initial_parameters.k

#define number of passive particles
n_passive_particles = initial_parameters.n_passive_particles

#define number of active particles
n_active_particles = initial_parameters.n_active_particles

#define drag-coefficient constant gamma
gamma = initial_parameters.gamma

#define rest distance between segments
r0 = initial_parameters.r0

#define temperature in units of kT
kT = initial_parameters.kT

#define integration step time
dt = initial_parameters.dt

#define number of integration steps
integration_steps = initial_parameters.integration_steps

#define number of points for graphical representation (number of callbacks)
number_of_points = initial_parameters.number_of_points

#define period for analysing of quantities (write them in data file)
return_period = integration_steps/number_of_points

In [4]:
#set epsilon and sigma for Lennard_jones potential between different particles
lj_epsilon_AA = initial_parameters.lj_epsilon_AA
lj_sigma_AA = initial_parameters.lj_sigma_AA
lj_epsilon_AB = initial_parameters.lj_epsilon_AB
lj_sigma_AB = initial_parameters.lj_sigma_AB
lj_epsilon_BB = initial_parameters.lj_epsilon_BB
lj_sigma_BB = initial_parameters.lj_sigma_BB

#set cut-off radius for potential between two particles (in units of lj_sigma_AA)
r_cut = initial_parameters.r_cut

In [5]:
#set path for results
name_for_same_simulations = initial_parameters.name_for_same_simulations

Results are saved for every single simulation.
therefore, the files (txt-file and gsd-file) have to have an own ID.
For example, each of 10 equal simulations (different seed) have ID's like
"simlation_1, ..., simulation_10". A new ID is made via a loop if an other ID
is already used.

In [6]:
#if a dat-file quantities_of_simulation_[ID] does not exist set simulation ID 
while True:
    #choose random simulation_id
    simulation_id = np.random.randint(0,99999)
    if os.path.isfile('Codes/results/'+str(name_for_same_simulations)+'/'+
                      'quantities_of_simulation_'+str(simulation_id)+'.dat'):
        pass

    else:
        
        break
        #unused simulation_id is created
        #(file for quantities is created in further steps automatically)

1


In [7]:
#set box_factor (x-, y- and z-length of simulation-box = box_factor * Monomers)
box_factor = initial_parameters.box_factor

# Create box with particles

In [8]:
#define box dimensions
Lx = box_factor * Monomers
Ly = box_factor * Monomers
Lz = box_factor * Monomers

#define the system as snapshot
snapshot = hoomd.data.make_snapshot(N = Monomers + n_passive_particles,
                                    
                                    #box-dimension: x-direction: -Lx/2 to Lx/2 etc.
                                    box=hoomd.data.boxdim(Lx=Lx, Ly=Ly, Lz=Lz),
                                    
                                    #list of types; typeid[0]='A', typeid[1]='B', etc.
                                    particle_types=['A','B'],
                                    bond_types=['polymer'])

# Set ID's positions and bonds for monomers/particles

In [9]:
#create list to set monomer's/particle's initial type-IDs, positions, bonds
initial_IDs = []
initial_positions = []
initial_bonds= []

#define initial quantities
initial_position_of_cm_x = 0
initial_position_of_cm_y = 0
initial_position_of_cm_z = 0

#set initial monomer positions and IDs; ATTENTION: particle with ID=0 is monomer,
#last monomer has ID Monomers-1; paricles with ID >= Monomers are not part of chain
for monomer in range(Monomers):
    initial_IDs.append(0)
    initial_positions.append([monomer - (Monomers/2) + 0.5, +Ly/2-1, +Lz/2-1])
    
    #calculate initial distance of center of mass from origin
    initial_position_of_cm_x += (initial_positions[monomer][0] / Monomers)
    initial_position_of_cm_y += (initial_positions[monomer][1] / Monomers)
    initial_position_of_cm_z += (initial_positions[monomer][2] / Monomers)
    
initial_position_of_cm = [initial_position_of_cm_x,
                          initial_position_of_cm_y,
                          initial_position_of_cm_z]

initial_end_to_end_vector = [initial_positions[Monomers-1][0]-initial_positions[0][0],
                             initial_positions[Monomers-1][1]-initial_positions[0][1],
                             initial_positions[Monomers-1][2]-initial_positions[0][2]]
    
for bond in range(Monomers-1):
    initial_bonds.append([bond, bond+1])
    
#resize the bonds list to actual number of bonds
snapshot.bonds.resize(Monomers-1)

#set initial passive particle positions and IDs, break loops if number of passiver particles is reached
passive_particles_counter = 0
for Lz_counter in range(-int(Lz/2-1), int(Lz/2-1)):
    
    if passive_particles_counter == n_passive_particles:
                break
            
    for Ly_counter in range(-int(Ly/2-1), int(Ly/2-1)):
        
        if passive_particles_counter == n_passive_particles:
                break  
                
        for Lx_counter in range(-int(Lx/2-1), int(Lx/2-1)):
            passive_particles_counter += 1
            initial_IDs.append(1)
            initial_positions.append([Lx_counter, Ly_counter, Lz_counter])
    
            if passive_particles_counter == n_passive_particles:
                break
    
#copy IDs, positions and bonds into valid arrays
snapshot.particles.typeid[:] = initial_IDs
snapshot.particles.position[:] = initial_positions
snapshot.bonds.group[:] = initial_bonds

# Initialization of integration parameters

In [10]:
#initialize HOOMD using snapshot
system = hoomd.init.read_snapshot(snapshot)

#set the bond type and strength between monomers
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=k, r0=r0)

#consider every particle
all = hoomd.group.all()

#safe the trajectory in 'results/datetime' to
#display in a visualisation program

#make only one gsd-file to safe memory for simulation with id=0
if simulation_id == 0:
    hoomd.dump.gsd('Codes/results/'+str(name_for_same_simulations)+'/'+
                   "trajectory_of_simulation_0.gsd",

                   #return_period = number of points in plots
                   period=return_period,
                   group=all, overwrite=True)

notice(2): Group "all" created containing 50 particles


# Set Lennard-Jones potential

In [ ]:
#cell method is used if r_cut is for all pairs of particles the same
nl = hoomd.md.nlist.cell()

#LJ-potential depens on r_cut
lj = hoomd.md.pair.lj(r_cut=r_cut, nlist=nl)

lj.pair_coeff.set('A', 'A', epsilon=lj_epsilon_AA, sigma=lj_sigma_AA)
lj.pair_coeff.set('A', 'B', epsilon=lj_epsilon_AB, sigma=lj_sigma_AB)
lj.pair_coeff.set('B', 'B', epsilon=lj_epsilon_BB, sigma=lj_sigma_BB)

# Set integrator and  parameters

In [11]:
#integrator
hoomd.md.integrate.mode_standard(dt=dt)
integrator = hoomd.md.integrate.brownian(group=all, kT=kT, dscale=False,
                                         seed=np.random.randint(0,9999))


#set gamma (friction constant); ATTENTION: set after integrator is set!
integrator.set_gamma('A', gamma=gamma)
integrator.set_gamma('B', gamma=gamma)


notice(2): integrate.langevin/bd is using specified gamma values


# Create classes to compute quantities

Note: For every run of THIS entire code ('run_polymer_simulation_once.ipynb') the current values for quantities are added to values calculated earlier. The division by the number of execution of THIS file is done by the 'main_code.ipynb'.

In [12]:
#create class for end-to-end distance
class Sq_end_to_end_distance:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        snapshot = self.system.take_snapshot()
        
        #ATTENTION: paricles.position[...] can also be position of passive particle!!!
        #paricles.position[0] (position 0) --> first monomer
        
        #find actual simulation box (if particle outside the box, it is in the next box), m=monomer
        m_first_image = snapshot.particles.image[0]
        m_last_image = snapshot.particles.image[Monomers-1]
        
        #calculate positions of first and last monomer Lx,Ly,Lz = length of simulation box
        m_first_x = snapshot.particles.position[0][0] + Lx*m_first_image[0]
        m_first_y = snapshot.particles.position[0][1] + Ly*m_first_image[1]
        m_first_z = snapshot.particles.position[0][2] + Lz*m_first_image[2]
        m_first = [m_first_x, m_first_y, m_first_z]
        
        m_last_x = snapshot.particles.position[Monomers-1][0] + Lx*m_last_image[0]
        m_last_y = snapshot.particles.position[Monomers-1][1] + Ly*m_last_image[1]
        m_last_z = snapshot.particles.position[Monomers-1][2] + Lz*m_last_image[2]  
        m_last = [m_last_x, m_last_y, m_last_z]
        
        #calculate squared end-to-end distance
        current_sq_end_to_end_distance = ((m_last[0]-m_first[0])**2 +
                                          (m_last[1]-m_first[1])**2 +
                                          (m_last[2]-m_first[2])**2)
        
        #return current end-to-end distance
        return (current_sq_end_to_end_distance)

In [13]:
#create class for distance of centre of mass from origin
class Sq_distance_of_cm:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        snapshot = self.system.take_snapshot()
        
        #define lists of x,y and z positions of monomers
        list_of_x_positions = []
        list_of_y_positions = []
        list_of_z_positions = []
        
        #compute current position of centre of mass (the average is done by other Python-program)
        for monomers in range(Monomers):
            
            #Particles can move outside the box (and appear in the same box on the
            #other side). Therefore, the actual position has
            #to be recalculated by unsing the box-length Lx, Ly, Lz
            list_of_x_positions.append(snapshot.particles.position[monomers][0]
                                      + Lx*snapshot.particles.image[monomers][0])
            list_of_y_positions.append(snapshot.particles.position[monomers][1]
                                      + Ly*snapshot.particles.image[monomers][1])
            list_of_z_positions.append(snapshot.particles.position[monomers][2]
                                      + Lz*snapshot.particles.image[monomers][2])
            
        current_position_of_cm = [np.mean(list_of_x_positions),
                                  np.mean(list_of_y_positions),
                                  np.mean(list_of_z_positions)]
        
        #calculation of squared distance of cm from origin
        current_sq_distance_of_cm = ((current_position_of_cm[0]-initial_position_of_cm[0])**2 +
                                     (current_position_of_cm[1]-initial_position_of_cm[1])**2 +
                                     (current_position_of_cm[2]-initial_position_of_cm[2])**2)
        
        #return current end-to-end distance        
        return (current_sq_distance_of_cm)

In [14]:
#create class for auto correlation of end-to-end distance vector
class Auto_corr_ee_vector:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        snapshot = self.system.take_snapshot()
        
        #ATTENTION: paricles.position[...] can also be position of passive particle!!!
        #paricles.position[0] (position 0) --> first monomer 
        
        current_end_to_end_vector = [(snapshot.particles.position[Monomers-1][0]
                                          + Lx*snapshot.particles.image[Monomers-1][0]) -
                                         
                                          (snapshot.particles.position[0][0]
                                          + Lx*snapshot.particles.image[0][0]),
                                        
                                    
                                        (snapshot.particles.position[Monomers-1][1]
                                          + Ly*snapshot.particles.image[Monomers-1][1]) -
                                         
                                          (snapshot.particles.position[0][1]
                                          + Ly*snapshot.particles.image[0][1]),
                                  
                                        
                                        (snapshot.particles.position[Monomers-1][2]
                                          + Lz*snapshot.particles.image[Monomers-1][2]) -
                                         
                                          (snapshot.particles.position[0][2]
                                          + Lz*snapshot.particles.image[0][2])]
        
        auto_corr_ee_vector = (current_end_to_end_vector[0] * initial_end_to_end_vector[0] +
                               current_end_to_end_vector[1] * initial_end_to_end_vector[1] +
                               current_end_to_end_vector[2] * initial_end_to_end_vector[2])
        
        #return current auto-corr end-to-end vector        
        return (auto_corr_ee_vector)

# Create instances for classes and log the quantities

In [15]:
instance_sq_end_to_end_distance = Sq_end_to_end_distance(system)
instance_sq_distance_of_cm      = Sq_distance_of_cm(system)
instance_auto_corr_ee_vector    = Auto_corr_ee_vector(system)

#use analyze.log to write quantities of simulation into a text file
logger = hoomd.analyze.log(filename=('Codes/results/'+str(name_for_same_simulations)+'/'+
                                     'quantities_of_simulation_'+
                                     str(simulation_id)+'.dat'),
                           quantities=['sq_end_to_end_distance',
                                       'sq_distance_of_cm',
                                       'auto_corr_ee_vector'],
                           
                           #return and write to file
                           period = return_period,
                           header_prefix='#',
                           overwrite=True)

#create a new quantity that is logged due to expression above to a text file
logger.register_callback(('sq_end_to_end_distance'), instance_sq_end_to_end_distance)
logger.register_callback(('sq_distance_of_cm'), instance_sq_distance_of_cm)
logger.register_callback(('auto_corr_ee_vector'), instance_auto_corr_ee_vector)

# Run the simulation

In [ ]:
hoomd.run(integration_steps)